# Exploração inicial dos dados

Objetivos deste notebook:
- Carregar as principais tabelas da pasta `data/`.
- Exibir resumos (shape, dtypes, amostras, estatísticas, missingness) de cada tabela.

Observação: este notebook não altera os dados; serve para diagnóstico e planejamento do pipeline de pré-processamento.

In [1]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 120)

# Diretório dos dados (ajustado para o projeto atual)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(project_root, 'data')
data_dir

'C:\\Users\\User\\PycharmProjects\\mestrado\\Machine Learning\\Project_1\\data'

Funções utilizadas durante o processo

In [141]:
import pandas as pd
from typing import Optional, Tuple, Any
import re
import unicodedata


def fill_forward(dataframe, column_name, new_column_name):
  """
  Fills missing values in a specified column by propagating the last valid
  observation forward. Works for dates, strings, and numbers.

  Args:
    dataframe (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to process.
    new_column_name (str): The name of the new column.

  Returns:
    pd.DataFrame: The DataFrame with the specified column cleaned.
  """
  if new_column_name is not None:
      dataframe[new_column_name] = dataframe[column_name].ffill()
      return dataframe
  # The 'ffill' method is all you need!
  dataframe[column_name].ffill(inplace=True)

  return dataframe

def fill_missing_dates(dataframe, column_name, new_column_name):
  """
  Fills missing values in a datetime column by forwarding the last valid observation.

  Args:
    dataframe (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to process.
    new_column_name (str): The name of the new column.

  Returns:
    pd.DataFrame: The DataFrame with the specified column cleaned.
  """
  # Convert the column to datetime objects, which turns blanks into NaT (Not a Time)
  dataframe[column_name] = pd.to_datetime(dataframe[column_name])

  # Use 'ffill' (forward fill) to propagate the last valid date forward
  if new_column_name is not None:
    dataframe[new_column_name] = dataframe[column_name].ffill()
    return dataframe
  dataframe[column_name].ffill(inplace=True)

  return dataframe

def _find_column_case_insensitive(df: pd.DataFrame, target: str) -> Optional[str]:
    """Return the actual column name matching `target` ignoring case and accents-like variations."""
    lower_map = {c.lower().strip(): c for c in df.columns}
    # common variations
    candidates = {
        target.lower(),
        target.replace("_", " ").lower(),
        target.replace(" ", "_").lower(),
    }
    for cand in candidates:
        if cand in lower_map:
            return lower_map[cand]
    # fallback: partial contains
    for k, v in lower_map.items():
        if target.lower().replace("_", " ") in k:
            return v
    return None

def transform_data_coleta_column(df: pd.DataFrame, column: str, in_place: bool = False, new_column: Optional[str] = None) -> Tuple[pd.DataFrame, Optional[str]]:
    """
    Transform the date values from the column to the string format dd/MM/AA.

    - If `in_place` is True, overwrites the detected column with the formatted string.
    - If `new_column` is provided, writes the formatted string to this new column instead.
      If both are provided, `new_column` takes precedence and original column is kept.

    Returns: (DataFrame, name_of_output_column)
    """
    if df is None or df.empty:
        return df, None

    col_name = _find_column_case_insensitive(df, column)
    if col_name is None:
        # nothing to do
        return df, None

    # Parse to datetime first, being tolerant with input
    dt = pd.to_datetime(df[col_name], errors="coerce", dayfirst=True, infer_datetime_format=True)

    if new_column:
        df[new_column] = dt
        out_col = new_column
    elif in_place:
        df[col_name] = dt
        out_col = col_name
    else:
        # default: create a new standardized name
        out_col = f"{col_name}_dt"
        df[out_col] = dt

    return df, out_col

def _flatten_column_name(col: Any) -> str:
    """Achata nomes de colunas possivelmente em MultiIndex e converte para string."""
    if isinstance(col, tuple):
        parts = [str(x) for x in col if x is not None and str(x) != '']
        return "_".join(parts)
    return str(col)


def _remove_accents(text: str) -> str:
    norm = unicodedata.normalize('NFKD', text)
    return ''.join(ch for ch in norm if not unicodedata.combining(ch))


def standardize_colname(name: Any) -> str:
    """
    Converte um nome de coluna para snake_case, minúsculas e sem acentos.
    - Remove acentos
    - Substitui caracteres não alfanuméricos por "_"
    - Colapsa underscores repetidos e remove underscores nas extremidades
    - Mantém apenas [a-z0-9_]
    - Lida com colunas em MultiIndex (provenientes de pivot)
    """
    s = _flatten_column_name(name)
    s = _remove_accents(s)
    # alguns símbolos comuns
    s = s.replace('%', 'percent').replace('°', '')
    # trocar qualquer coisa que não seja letra/numero por underscore
    s = re.sub(r'[^0-9a-zA-Z]+', '_', s)
    # colapsar e aparar
    s = re.sub(r'_+', '_', s).strip('_')
    return s.lower()


def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Retorna uma cópia do DataFrame com os nomes de colunas padronizados."""
    if df is None:
        return df
    out = df.copy()
    out.columns = [standardize_colname(c) for c in out.columns]
    return out


Como foi realizada a coleta dos dados:

Foi realizado um estudo em uma plantação de milho, onde foi utilizado dois tipos de plantação em disco (D) e plantio direto (PD), utilizando 3 tipos de genótipos diferentes em cada uma das plantação, denotadas por G1,G2 e G3.

Durante todo o periodo do crescimento e da colheita do milho foram avaliados algumas caracteristicas e valores.

Para determinar se a planta passou por um periodo de estress é utilizado o seguinte cálculo:

Fazer a diferença entre os valores de PMP (Ponto de murcha permanente) e CC (Capacidade de Campo), se a Umidade Relativa do solo atingir 50% deste valor da diferença, considera que a planta esteve sob estresse.

Baseado na tabela enviada dos dados

![tabela_de_pmp_e_cc](../docs/tabela_de_PMP_e_CC.png)

A linha 71 se refere aos dados da plantação em disco e a linha 60 ao Plantio Direto

Portanto para o Disco, o limite de umidade seria = 0.362 - 0.243 = 0.119 e 50% desse valor seria 0.0595

E para Plantio direto seria = 0.407 - 0.291 = 0.116 e 50% desse valor seria 0.058

O primeiro passo para a análise dos dados é ler as tabelas e avaliar os comportamentos.

In [131]:
list_where_second_line_is_header= ["Espigas.xlsx", "Altura plantas.xlsx", "Fenologia inicial.xlsx", "Fenologia completa.xlsx"]
list_where_forth_line_is_header = ["Pesos.xlsx"]
list_where_first_line_is_header = ["Umidade de graos para maturidade.xlsx", "Maturidade fisiologica.xlsx"]
def read_excel_safe(path):
    try:
        # If your using linux change this split for the "/"
        if path.split("\\")[-1] in list_where_second_line_is_header:
            return pd.read_excel(path, header=2)
        if path.split("\\")[-1] in list_where_forth_line_is_header:
            return pd.read_excel(path, header=3)
        if path.split("\\")[-1] in list_where_first_line_is_header:
            return pd.read_excel(path, header=1)
        return pd.read_excel(path, header=True)
    except Exception as e1:
        try:
            return pd.read_excel(path, engine='openpyxl')
        except Exception as e2:
            print(f'Falha ao ler {path}:', e1, '|', e2)
            return None

files = {
    'Espigas': 'Espigas.xlsx',
    'Dados_de_colheita': 'Dados de colheita.xlsx',
    'Umidade_de_solo': 'Umidade de solo.xlsx',
    'Umidade_de_graos_maturidade': 'Umidade de graos para maturidade.xlsx',
    'Umidade_a_partir_do_ponto_pamonha': 'Umidade a partir do ponto pamonha.xlsx',
    'Altura_plantas': 'Altura plantas.xlsx',
    'Pesos': 'Pesos.xlsx',
    'Fenologia_inicial': 'Fenologia inicial.xlsx',
    'Fenologia_completa': 'Fenologia completa.xlsx',
    'Maturidade_fisiologica': 'Maturidade fisiologica.xlsx'
}

loaded = {}
for name, fname in files.items():
    fpath = os.path.join(data_dir, fname)
    if not os.path.exists(fpath):
        print(f'[AVISO] Arquivo não encontrado: {fpath}')
        loaded[name] = None
        continue
    df = read_excel_safe(fpath)
    loaded[name] = df
    #summarize_df(name, df)

In [26]:
loaded["Espigas"]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Faixa,Parcela,Tratamento,Genotipo,Repetiçao,Terço_inferior_mm,Terço_medio_mm,Terço_superior_mm,Numero_fileiras,Numero_graos_1_fileira,Comprimento_espiga_mm
2,1,104,Disco,G1,R1,50.1,48.2,39.8,16,32,145
3,1,104,Disco,G2,R1,48,50.8,44.9,18,41,175
4,1,104,Disco,G3,R1,54.9,56.5,48,20,36,155
5,1,107,Plantio Direto,G1,R1,53.4,54,43.1,20,30,170
6,1,107,Plantio Direto,G2,R1,48.7,49.2,44.8,20,36,160
7,1,107,Plantio Direto,G3,R1,54.2,55.7,45.5,18,36,160
8,2,204,Plantio Direto,G1,R2,49.6,48,40.4,18,36,155
9,2,204,Plantio Direto,G2,R2,48.3,50.4,38.9,18,40,185


Mesmo considerando como para o algoritmo ler o arquivo utilizando a primeira linha como header, alguns arquivos aparentam possuir a segunda linha como header, sendo necessário pular uma linha para alguns casos.

In [40]:
loaded["Espigas"]

,Faixa,Parcela,Tratamento,Genotipo,Repetiçao,Terço_inferior_mm,Terço_medio_mm,Terço_superior_mm,Numero_fileiras,Numero_graos_1_fileira,Comprimento_espiga_mm
0,1,104,Disco,G1,R1,50.1,48.2,39.8,16,32,145
1,1,104,Disco,G2,R1,48.0,50.8,44.9,18,41,175
2,1,104,Disco,G3,R1,54.9,56.5,48.0,20,36,155
3,1,107,Plantio Direto,G1,R1,53.4,54.0,43.1,20,30,170
4,1,107,Plantio Direto,G2,R1,48.7,49.2,44.8,20,36,160
5,1,107,Plantio Direto,G3,R1,54.2,55.7,45.5,18,36,160
6,2,204,Plantio Direto,G1,R2,49.6,48.0,40.4,18,36,155
7,2,204,Plantio Direto,G2,R2,48.3,50.4,38.9,18,40,185
8,2,204,Plantio Direto,G3,R2,52.0,53.1,46.4,18,32,140
9,2,211,Disco,G1,R2,50.2,48.8,42.5,18,36,155


Agora o dataframe está de acordo com as colunas do xlsx

In [41]:
loaded["Dados_de_colheita"]

,Faixa,Genotipo,Parcela,Peso_total_kg,Peso_1000_graos_g,Umidade_porcentagem,Estande_final_2_linhas,Parcela.1,Produtividade_corrigida_umidade_13_por_cento,Produtividade_hectare,Media_produtividade_hectare
0,1,G1,101,20.35,260.6,14.0,121,101,20.116092,7184.318555,9460.985222
1,1,G1,102,29.49,323.7,13.1,140,102,29.456103,10520.036946,NaN
2,1,G1,103,21.24,285.5,12.5,146,103,21.362069,7629.310345,NaN
3,1,G1,104,24.63,306.6,13.0,140,104,24.630000,8796.428571,NaN
4,1,G1,105,20.97,300.3,12.6,157,105,21.066414,7523.719212,NaN
...,...,...,...,...,...,...,...,...,...,...,...
166,6,G3,1904,23.82,247.4,14.4,159,1904,23.436690,8370.246305,NaN
167,6,G3,1905,24.19,200.5,13.2,143,1905,24.134391,8619.425287,NaN
168,6,G3,1906,23.63,244.8,13.2,142,1906,23.575678,8419.885057,NaN
169,6,G3,1907,26.76,215.7,12.4,160,1907,26.944552,9623.054187,NaN


In [42]:
loaded["Umidade_de_solo"]

,Profundidade_cm,Registro,Identificacao,Data_coleta,Material,Umidade_porcentagem,Conteudo_gravimetrico_agua_kgkg1
0,0-30,425757,Solo D1,21/12/2023,SOLO,25.79,0.2579
1,30-60,425758,Solo D2,21/12/2023,SOLO,27.92,0.2792
2,60-90,425759,Solo D3,21/12/2023,SOLO,29.30,0.2930
3,90-120,425760,Solo D4,21/12/2023,SOLO,28.89,0.2889
4,0-30,425761,Solo PD1,21/12/2023,SOLO,27.63,0.2763
5,30-60,425762,Solo PD2,21/12/2023,SOLO,30.56,0.3056
6,60-90,425763,Solo PD3,21/12/2023,SOLO,30.20,0.3020
7,90-120,425764,Solo PD4,21/12/2023,SOLO,29.78,0.2978
8,0-30,425765,Solo D1,28/12/2023,SOLO,27.22,0.2722
9,30-60,425766,Solo D2,2023-12-28 00:00:00,SOLO,26.94,0.2694


Os seguintes arquivos também possuem a segunda linha como header.

In [43]:
loaded["Umidade_de_graos_maturidade"]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Tratamento,Coleta,Data_entrada,Data_saida,Peso_pote,Peso_pote_e_milho_umido,Peso_pote_e_milho_seco,Umidade_porcentagem
1,G1 PLANTIO DIRETO,1,2024-04-01 00:00:00,2024-04-02 00:00:00,32.37,78.85,70.08,18.86833
2,G1 DISCO,1,2024-04-01 00:00:00,2024-04-02 00:00:00,31.87,76.15,67.33,19.918699
3,G2 PLANTIO DIRETO,1,2024-04-01 00:00:00,2024-04-02 00:00:00,32.01,73.82,65.77,19.253767
4,G2 DISCO,1,2024-04-01 00:00:00,2024-04-02 00:00:00,32.21,68.81,61.05,21.202186
5,G3 PLANTIO DIRETO,1,2024-04-01 00:00:00,2024-04-02 00:00:00,32.28,75.76,67.58,18.813247
6,G3 DISCO,1,2024-04-01 00:00:00,2024-04-02 00:00:00,32.4,69.36,62.06,19.751082
7,G1 PLANTIO DIRETO,2,2024-04-08 00:00:00,2024-04-09 00:00:00,32.38,72.26,66.36,14.794383
8,G1 DISCO,2,2024-04-08 00:00:00,2024-04-09 00:00:00,31.89,68.11,62.4,15.764771
9,G2 PLANTIO DIRETO,2,2024-04-08 00:00:00,2024-04-09 00:00:00,32,71.33,64.92,16.297991


In [44]:
loaded["Altura_plantas"]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Faixa,Parcela,Tratamento,Genotipo,Repetiçao,Altura_planta_1,Altura_planta_2,Altura_planta_3,Altura_planta_4,Altura_planta_5,Altura_espiga_1,Altura_espiga_2,Altura_espiga_3,Altura_espiga_4,Altura_espiga_5
2,1,104,Disco,G1,R1,250,270,290,285,285,130,140,160,125,145
3,1,104,Disco,G2,R1,265,260,260,265,265,145,115,120,130,115
4,1,104,Disco,G3,R1,300,310,310,310,295,140,135,140,145,135
5,1,107,Plantio Direto,G1,R1,270,290,295,295,295,160,165,170,165,175
6,1,107,Plantio Direto,G2,R1,285,300,305,290,295,160,165,175,135,155
7,1,107,Plantio Direto,G3,R1,340,330,330,330,335,170,165,195,165,165
8,2,204,Plantio Direto,G1,R2,280,295,285,270,290,145,155,165,160,165
9,2,204,Plantio Direto,G2,R2,300,280,295,270,270,140,135,170,145,130


In [45]:
loaded["Pesos"]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tara: 22.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Preparo_solo,Parcela,Identificacao,Plantas_por_metro,Peso_verde_5_plantas,Peso_verde_colmo_5_plantas,Peso_seco_colmo_5_plantas,Peso_verde_folhas_5_plantas,Peso_seco_folhas_5_plantas,Numero_de_espigas,Peso_verde_espigas,Peso_espiga_com_graos,Peso_verde_palha_espiga,Peso_seco_palha_espiga,Peso_verde_sabugo_espiga,Peso_seco_sabugo_espiga,Peso_verde_grao_espiga,Peso_seco_grao_espiga,Numero_bonecas,Peso_verde_bonecas,Peso_verde_palha_boneca,Peso_seco_palha_boneca,Peso_verde_sabugo_boneca,Peso_seco_sabugo_boneca,Peso_verde_graos_boneca,Peso_seco_graos_boneca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Disco,104,45393,5,3475,1796.36,518.8,351.92,356.92,5,1498.3,1354.68,141.39,121.64,182.15,133.13,1126.62,1009.13,1,2.87,2.87,1.45,9999,9999,9999,9999,NaN,Preparo_solo,Parcela,Identificacao,Plantas_por_metro,Peso_verde_plantas,Peso_colmo_plantas,Peso_folhas_plantas,Area_foliar_plantas_cm2,Peso_verde_folhas_aliquota,Peso_seco_folhas_aliquota,Peso_verde_F_maduras,Peso_seco_F_maduras,Peso_verde_colmo_aliquota,Peso_seco_colmo_aliquota,Numero_espigas,Peso_verde_espiga,Peso_espiga_com_graos,Peso_verde_palha_da_espiga,Peso_seco_palha_da_espiga,Peso_verde_sabugo,Peso_seco_sabugo,Peso_verde_graos,Peso_seco_graos,Numero_bonecas,Peso_verde_bonecas,Peso_verde_palha_boneca,Peso_seco_palha_boneca,Peso_verde_sabugo_boneca,Peso_seco_sabugo_boneca
4,NaN,211,45393,5,2745,1174.09,438.61,311.81,311.81,5,1225.85,1101.13,122.08,104.99,142.61,114.22,957.04,831.15,3,17.11,13.8,1.98,3.52,2.1,9999,9999,NaN,Disco,104,45279,5,1700,1021.98,690.42,18009.6,355,97.6,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999
5,NaN,309,45393,5,3390,1514.31,507.22,352.45,352.45,5,1485.47,1336.19,145.96,121.79,187.52,142.15,1150.62,995.19,1,2.14,2.14,0.91,9999,9999,9999,9999,NaN,NaN,NaN,45303,5,4440,3124,1052,39622.51,330.53,57.53,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999
6,Plantio Direto,107,45393,5,3065,1577.9,408.15,324.24,324.24,5,1134.39,1014.56,117.55,94.2,113.62,101.21,883.13,760.05,1,1.76,1.76,0.1,9999,9999,9999,9999,NaN,NaN,NaN,45348,5,6790,2920,885,35619.87,885,276.16,47.45,36.86,1130,280.4,6,2358.73,1882.12,442.63,148.59,426.64,167.87,1440.89,770.37,2,38.8,40.37,7.53,10.89,0.66
7,NaN,204,45393,5,2970,1246.17,364.62,302.8,302.8,5,1402.43,1291.98,108.04,93.08,163.38,120.24,1126.76,968.43,1,1.48,1.48,0.01,9999,9999,9999,9999,NaN,NaN,211,45279,5,1800,1134,708.19,16798.92,421.01,88.3,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999
8,NaN,310,45393,5,2730,1158.13,436.12,228.96,430.41,5,1325.06,1209.44,113.39,107.11,164.79,128.61,1045.67,903.36,2,5.75,5.75,3.79,9999,9999,9999,9999,NaN,NaN,NaN,45303,5,5040,3140,1090,46643.47,358.96,71.31,9999,9999,9999,9999,9

In [46]:
loaded["Fenologia_inicial"]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data,Faixa,Parcela,Tratamento,Repeticao,Planta,Folhas_verdadeiras,Folhas_totais
2,2023-12-14 00:00:00,1,1,Disco,R1,1,6,12
3,NaN,NaN,NaN,NaN,NaN,2,7,12
4,NaN,NaN,NaN,NaN,NaN,3,8,13
...,...,...,...,...,...,...,...,...
137,NaN,NaN,NaN,NaN,R2,6,18,20
138,NaN,NaN,NaN,NaN,NaN,7,17,19
139,NaN,NaN,NaN,NaN,NaN,8,18,20
140,NaN,NaN,NaN,NaN,NaN,9,17,19


In [47]:
loaded["Fenologia_completa"]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data,Faixa,Parcela,Tratamento,Genotipo,Repeticao,Planta,FV,FT,FB,Boneca,NaN
2,2024-01-15 00:00:00,1,1,Disco,NaN,R1,1,20,20,OK,*,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,2,20,21,NaN,*,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,3,21,23,NaN,*,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
77,NaN,NaN,NaN,NaN,NaN,R2,6,-,-,-,-,OK
78,NaN,NaN,NaN,NaN,NaN,NaN,7,-,-,-,-,OK
79,NaN,NaN,NaN,NaN,NaN,NaN,8,-,-,-,-,OK
80,NaN,NaN,NaN,NaN,NaN,NaN,9,-,-,-,-,-


In [78]:
loaded["Maturidade_fisiologica"]

,Data,Faixa,Parcela,Tratamento,Genotipo,Repeticao,Numero_graos,Numero_graos_maduros
0,2024-02-29,1,104,Disco,G1,1,30,0
1,NaT,1,104,Disco,G2,1,30,0
2,NaT,1,104,Disco,G3,1,30,0
3,NaT,1,107,Plantio Direto,G1,1,30,0
4,NaT,1,107,Plantio Direto,G2,1,30,0
...,...,...,...,...,...,...,...,...
211,NaT,3,309,Disco,G2,3,60,60
212,NaT,3,309,Disco,G3,3,60,60
213,NaT,3,310,Plantio Direto,G1,3,60,60
214,NaT,3,310,Plantio Direto,G2,3,60,60


Após a modificação na leitura

In [79]:
loaded["Umidade_de_graos_maturidade"]

,Tratamento,Coleta,Data_entrada,Data_saida,Peso_pote,Peso_pote_e_milho_umido,Peso_pote_e_milho_seco,Umidade_porcentagem
0,G1 PLANTIO DIRETO,1,2024-04-01,2024-04-02,32.37,78.85,70.08,18.868330
1,G1 DISCO,1,2024-04-01,2024-04-02,31.87,76.15,67.33,19.918699
2,G2 PLANTIO DIRETO,1,2024-04-01,2024-04-02,32.01,73.82,65.77,19.253767
3,G2 DISCO,1,2024-04-01,2024-04-02,32.21,68.81,61.05,21.202186
4,G3 PLANTIO DIRETO,1,2024-04-01,2024-04-02,32.28,75.76,67.58,18.813247
5,G3 DISCO,1,2024-04-01,2024-04-02,32.40,69.36,62.06,19.751082
6,G1 PLANTIO DIRETO,2,2024-04-08,2024-04-09,32.38,72.26,66.36,14.794383
7,G1 DISCO,2,2024-04-08,2024-04-09,31.89,68.11,62.40,15.764771
8,G2 PLANTIO DIRETO,2,2024-04-08,2024-04-09,32.00,71.33,64.92,16.297991
9,G2 DISCO,2,2024-04-08,2024-04-09,32.22,68.82,62.59,17.021858


Aqui o header acabou ficando no dado, sendo necessário avaliar novamente.

In [73]:
loaded["Umidade_de_graos_maturidade"]

,Tratamento,Coleta,Data_entrada,Data_saida,Peso_pote,Peso_pote_e_milho_umido,Peso_pote_e_milho_seco,Umidade_porcentagem
0,G1 PLANTIO DIRETO,1,2024-04-01,2024-04-02,32.37,78.85,70.08,18.868330
1,G1 DISCO,1,2024-04-01,2024-04-02,31.87,76.15,67.33,19.918699
2,G2 PLANTIO DIRETO,1,2024-04-01,2024-04-02,32.01,73.82,65.77,19.253767
3,G2 DISCO,1,2024-04-01,2024-04-02,32.21,68.81,61.05,21.202186
4,G3 PLANTIO DIRETO,1,2024-04-01,2024-04-02,32.28,75.76,67.58,18.813247
5,G3 DISCO,1,2024-04-01,2024-04-02,32.40,69.36,62.06,19.751082
6,G1 PLANTIO DIRETO,2,2024-04-08,2024-04-09,32.38,72.26,66.36,14.794383
7,G1 DISCO,2,2024-04-08,2024-04-09,31.89,68.11,62.40,15.764771
8,G2 PLANTIO DIRETO,2,2024-04-08,2024-04-09,32.00,71.33,64.92,16.297991
9,G2 DISCO,2,2024-04-08,2024-04-09,32.22,68.82,62.59,17.021858


In [51]:
loaded["Altura_plantas"]

,Faixa,Parcela,Tratamento,Genotipo,Repetiçao,Altura_planta_1,Altura_planta_2,Altura_planta_3,Altura_planta_4,Altura_planta_5,Altura_espiga_1,Altura_espiga_2,Altura_espiga_3,Altura_espiga_4,Altura_espiga_5
0,1,104,Disco,G1,R1,250,270,290,285,285,130,140,160,125,145
1,1,104,Disco,G2,R1,265,260,260,265,265,145,115,120,130,115
2,1,104,Disco,G3,R1,300,310,310,310,295,140,135,140,145,135
3,1,107,Plantio Direto,G1,R1,270,290,295,295,295,160,165,170,165,175
4,1,107,Plantio Direto,G2,R1,285,300,305,290,295,160,165,175,135,155
5,1,107,Plantio Direto,G3,R1,340,330,330,330,335,170,165,195,165,165
6,2,204,Plantio Direto,G1,R2,280,295,285,270,290,145,155,165,160,165
7,2,204,Plantio Direto,G2,R2,300,280,295,270,270,140,135,170,145,130
8,2,204,Plantio Direto,G3,R2,340,340,345,345,330,180,180,190,190,155
9,2,211,Disco,G1,R2,245,280,285,245,260,135,155,155,150,135


In [65]:
loaded["Pesos"]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55
0,Preparo_solo,Parcela,Identificacao,Plantas_por_metro,Peso_verde_5_plantas,Peso_verde_colmo_5_plantas,Peso_seco_colmo_5_plantas,Peso_verde_folhas_5_plantas,Peso_seco_folhas_5_plantas,Numero_de_espigas,Peso_verde_espigas,Peso_espiga_com_graos,Peso_verde_palha_espiga,Peso_seco_palha_espiga,Peso_verde_sabugo_espiga,Peso_seco_sabugo_espiga,Peso_verde_grao_espiga,Peso_seco_grao_espiga,Numero_bonecas,Peso_verde_bonecas,Peso_verde_palha_boneca,Peso_seco_palha_boneca,Peso_verde_sabugo_boneca,Peso_seco_sabugo_boneca,Peso_verde_graos_boneca,Peso_seco_graos_boneca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Disco,104,45393,5,3475,1796.36,518.8,351.92,356.92,5,1498.3,1354.68,141.39,121.64,182.15,133.13,1126.62,1009.13,1,2.87,2.87,1.45,9999,9999,9999,9999,NaN,Preparo_solo,Parcela,Identificacao,Plantas_por_metro,Peso_verde_plantas,Peso_colmo_plantas,Peso_folhas_plantas,Area_foliar_plantas_cm2,Peso_verde_folhas_aliquota,Peso_seco_folhas_aliquota,Peso_verde_F_maduras,Peso_seco_F_maduras,Peso_verde_colmo_aliquota,Peso_seco_colmo_aliquota,Numero_espigas,Peso_verde_espiga,Peso_espiga_com_graos,Peso_verde_palha_da_espiga,Peso_seco_palha_da_espiga,Peso_verde_sabugo,Peso_seco_sabugo,Peso_verde_graos,Peso_seco_graos,Numero_bonecas,Peso_verde_bonecas,Peso_verde_palha_boneca,Peso_seco_palha_boneca,Peso_verde_sabugo_boneca,Peso_seco_sabugo_boneca
2,NaN,211,45393,5,2745,1174.09,438.61,311.81,311.81,5,1225.85,1101.13,122.08,104.99,142.61,114.22,957.04,831.15,3,17.11,13.8,1.98,3.52,2.1,9999,9999,NaN,Disco,104,45279,5,1700,1021.98,690.42,18009.6,355,97.6,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999
3,NaN,309,45393,5,3390,1514.31,507.22,352.45,352.45,5,1485.47,1336.19,145.96,121.79,187.52,142.15,1150.62,995.19,1,2.14,2.14,0.91,9999,9999,9999,9999,NaN,NaN,NaN,45303,5,4440,3124,1052,39622.51,330.53,57.53,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999
4,Plantio Direto,107,45393,5,3065,1577.9,408.15,324.24,324.24,5,1134.39,1014.56,117.55,94.2,113.62,101.21,883.13,760.05,1,1.76,1.76,0.1,9999,9999,9999,9999,NaN,NaN,NaN,45348,5,6790,2920,885,35619.87,885,276.16,47.45,36.86,1130,280.4,6,2358.73,1882.12,442.63,148.59,426.64,167.87,1440.89,770.37,2,38.8,40.37,7.53,10.89,0.66
5,NaN,204,45393,5,2970,1246.17,364.62,302.8,302.8,5,1402.43,1291.98,108.04,93.08,163.38,120.24,1126.76,968.43,1,1.48,1.48,0.01,9999,9999,9999,9999,NaN,NaN,211,45279,5,1800,1134,708.19,16798.92,421.01,88.3,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999
6,NaN,310,45393,5,2730,1158.13,436.12,228.96,430.41,5,1325.06,1209.44,113.39,107.11,164.79,128.61,1045.67,903.36,2,5.75,5.75,3.79,9999,9999,9999,9999,NaN,NaN,NaN,45303,5,5040,3140,1090,46643.47,358.96,71.31,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45348,5,6020,2450,720,28992.35,720,247.5,21.85,18.72,785.33,209.04,5,2142.94,1787.49,323.55,124.5,399.34,163.02,1377.69,744.58,3,99.41,58.88,12.12,16,1.5
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

Essa tabela em específico, o header estava na quarta linha.

In [67]:
loaded["Pesos"]

,Preparo_solo,Parcela,Identificacao,Plantas_por_metro,Peso_verde_5_plantas,Peso_verde_colmo_5_plantas,Peso_seco_colmo_5_plantas,Peso_verde_folhas_5_plantas,Peso_seco_folhas_5_plantas,Numero_de_espigas,Peso_verde_espigas,Peso_espiga_com_graos,Peso_verde_palha_espiga,Peso_seco_palha_espiga,Peso_verde_sabugo_espiga,Peso_seco_sabugo_espiga,Peso_verde_grao_espiga,Peso_seco_grao_espiga,Numero_bonecas,Peso_verde_bonecas,Peso_verde_palha_boneca,Peso_seco_palha_boneca,Peso_verde_sabugo_boneca,Peso_seco_sabugo_boneca,Peso_verde_graos_boneca,Peso_seco_graos_boneca,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55
0,Disco,104.0,45393.0,5.0,3475.0,1796.36,518.80,351.92,356.92,5.0,1498.30,1354.68,141.39,121.64,182.15,133.13,1126.62,1009.13,1.0,2.87,2.87,1.45,9999.00,9999.0,9999.0,9999.0,NaN,Preparo_solo,Parcela,Identificacao,Plantas_por_metro,Peso_verde_plantas,Peso_colmo_plantas,Peso_folhas_plantas,Area_foliar_plantas_cm2,Peso_verde_folhas_aliquota,Peso_seco_folhas_aliquota,Peso_verde_F_maduras,Peso_seco_F_maduras,Peso_verde_colmo_aliquota,Peso_seco_colmo_aliquota,Numero_espigas,Peso_verde_espiga,Peso_espiga_com_graos,Peso_verde_palha_da_espiga,Peso_seco_palha_da_espiga,Peso_verde_sabugo,Peso_seco_sabugo,Peso_verde_graos,Peso_seco_graos,Numero_bonecas,Peso_verde_bonecas,Peso_verde_palha_boneca,Peso_seco_palha_boneca,Peso_verde_sabugo_boneca,Peso_seco_sabugo_boneca
1,NaN,211.0,45393.0,5.0,2745.0,1174.09,438.61,311.81,311.81,5.0,1225.85,1101.13,122.08,104.99,142.61,114.22,957.04,831.15,3.0,17.11,13.80,1.98,3.52,2.1,9999.0,9999.0,NaN,Disco,104,45279,5,1700,1021.98,690.42,18009.6,355,97.6,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999
2,NaN,309.0,45393.0,5.0,3390.0,1514.31,507.22,352.45,352.45,5.0,1485.47,1336.19,145.96,121.79,187.52,142.15,1150.62,995.19,1.0,2.14,2.14,0.91,9999.00,9999.0,9999.0,9999.0,NaN,NaN,NaN,45303,5,4440,3124,1052,39622.51,330.53,57.53,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999
3,Plantio Direto,107.0,45393.0,5.0,3065.0,1577.90,408.15,324.24,324.24,5.0,1134.39,1014.56,117.55,94.20,113.62,101.21,883.13,760.05,1.0,1.76,1.76,0.10,9999.00,9999.0,9999.0,9999.0,NaN,NaN,NaN,45348,5,6790,2920,885,35619.87,885,276.16,47.45,36.86,1130,280.4,6,2358.73,1882.12,442.63,148.59,426.64,167.87,1440.89,770.37,2,38.8,40.37,7.53,10.89,0.66
4,NaN,204.0,45393.0,5.0,2970.0,1246.17,364.62,302.80,302.80,5.0,1402.43,1291.98,108.04,93.08,163.38,120.24,1126.76,968.43,1.0,1.48,1.48,0.01,9999.00,9999.0,9999.0,9999.0,NaN,NaN,211,45279,5,1800,1134,708.19,16798.92,421.01,88.3,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999
5,NaN,310.0,45393.0,5.0,2730.0,1158.13,436.12,228.96,430.41,5.0,1325.06,1209.44,113.39,107.11,164.79,128.61,1045.67,903.36,2.0,5.75,5.75,3.79,9999.00,9999.0,9999.0,9999.0,NaN,NaN,NaN,45303,5,5040,3140,1090,46643.47,358.96,71.31,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45348,5,6020,2450,720,28992.35,720,247.5,21.85,18.72,785.33,209.04,5,2142.94,1787.49,323.55,124.5,399.34,163.02,1377.69,744.58,3,99.41,58.88,12.12,16,1.5
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,309,45279,5,733.02,389.64,342.37,8157.76,342.37,48.4,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45303,5,4420,3080,1044,41577

Mesmo após arrumar o header, pode-se notar que a partir da coluna 26 os mesmos campos aparecem novamente, como se duas tabelas diferentes estivessem no mesmo arquivo. Ao entrar no arquivo para avaliação.

![Two_tables_pesos](../docs/two_tables_pesos.png)

Há diferente tabelas no mesmo arquivo com número de colunas distintas, o que necessita de uma análise cuidadosa e com o especialista no assunto.

In [68]:
loaded["Fenologia_inicial"]

,Data,Faixa,Parcela,Tratamento,Repeticao,Planta,Folhas_verdadeiras,Folhas_totais
0,2023-12-14,1.0,1.0,Disco,R1,1,6,12
1,NaT,NaN,NaN,NaN,NaN,2,7,12
2,NaT,NaN,NaN,NaN,NaN,3,8,13
3,NaT,NaN,NaN,NaN,NaN,4,7,13
4,NaT,NaN,NaN,NaN,NaN,5,8,14
...,...,...,...,...,...,...,...,...
135,NaT,NaN,NaN,NaN,R2,6,18,20
136,NaT,NaN,NaN,NaN,NaN,7,17,19
137,NaT,NaN,NaN,NaN,NaN,8,18,20
138,NaT,NaN,NaN,NaN,NaN,9,17,19


In [76]:
loaded["Fenologia_completa"]

,Data,Faixa,Parcela,Tratamento,Genotipo,Repeticao,Planta,FV,FT,FB,Boneca,Unnamed: 11
0,2024-01-15,1.0,1.0,Disco,NaN,R1,1,20,20,OK,*,NaN
1,NaT,NaN,NaN,NaN,NaN,NaN,2,20,21,NaN,*,NaN
2,NaT,NaN,NaN,NaN,NaN,NaN,3,21,23,NaN,*,NaN
3,NaT,NaN,NaN,NaN,NaN,NaN,4,21,22,NaN,*,NaN
4,NaT,NaN,NaN,NaN,NaN,NaN,5,21,22,NaN,*,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
75,NaT,NaN,NaN,NaN,NaN,R2,6,-,-,-,-,OK
76,NaT,NaN,NaN,NaN,NaN,NaN,7,-,-,-,-,OK
77,NaT,NaN,NaN,NaN,NaN,NaN,8,-,-,-,-,OK
78,NaT,NaN,NaN,NaN,NaN,NaN,9,-,-,-,-,-


In [75]:
loaded["Maturidade_fisiologica"]

,2024-02-29 00:00:00,1,104,Disco,G1,1.1,30,0
0,NaT,1,104,Disco,G2,1,30,0
1,NaT,1,104,Disco,G3,1,30,0
2,NaT,1,107,Plantio Direto,G1,1,30,0
3,NaT,1,107,Plantio Direto,G2,1,30,0
4,NaT,1,107,Plantio Direto,G3,1,30,0
...,...,...,...,...,...,...,...,...
210,NaT,3,309,Disco,G2,3,60,60
211,NaT,3,309,Disco,G3,3,60,60
212,NaT,3,310,Plantio Direto,G1,3,60,60
213,NaT,3,310,Plantio Direto,G2,3,60,60


Avaliando as colunas das tabelas, há algumas tabelas que possuem a data em formato dd/MM/YY e outra em timestamp, enquanto outra não possuem a data, pois são casos em que a data é expandida para outras linhas, como pode ser visto no caso abaixo, portanto é necessário transformar todas em datetime para tratar como data.

Exemplo abaixo é da tabela de maturidade_fisiologica, onde uma linha é considerada como muitas para não precisar repetir o mesmo valor, porém ao trabalhar com csv ela acaba trazendo esses campos como nulo.

Começando com a Umidade_de_solo, vamos transformar a data para um único tipo (datetime)

![maturidade_fisiologica_table](../docs/maturidade_fisiologica.png)

In [132]:
transform_data_coleta_column(loaded["Umidade_de_solo"], "Data_coleta", False, "data_coleta_fmt")
loaded["Umidade_de_solo"]

C:\Users\User\AppData\Local\Temp\ipykernel_7856\3617860413.py:86: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dt = pd.to_datetime(df[col_name], errors="coerce", dayfirst=True, infer_datetime_format=True)


,Profundidade_cm,Registro,Identificacao,Data_coleta,Material,Umidade_porcentagem,Conteudo_gravimetrico_agua_kgkg1,data_coleta_fmt
0,0-30,425757,Solo D1,21/12/2023,SOLO,25.79,0.2579,2023-12-21
1,30-60,425758,Solo D2,21/12/2023,SOLO,27.92,0.2792,2023-12-21
2,60-90,425759,Solo D3,21/12/2023,SOLO,29.30,0.2930,2023-12-21
3,90-120,425760,Solo D4,21/12/2023,SOLO,28.89,0.2889,2023-12-21
4,0-30,425761,Solo PD1,21/12/2023,SOLO,27.63,0.2763,2023-12-21
5,30-60,425762,Solo PD2,21/12/2023,SOLO,30.56,0.3056,2023-12-21
6,60-90,425763,Solo PD3,21/12/2023,SOLO,30.20,0.3020,2023-12-21
7,90-120,425764,Solo PD4,21/12/2023,SOLO,29.78,0.2978,2023-12-21
8,0-30,425765,Solo D1,28/12/2023,SOLO,27.22,0.2722,2023-12-28
9,30-60,425766,Solo D2,2023-12-28 00:00:00,SOLO,26.94,0.2694,2023-12-28


In [101]:
loaded["Umidade_de_solo"].dtypes

Profundidade_cm                             object
Registro                                     int64
Identificacao                               object
Data_coleta                                 object
Material                                    object
Umidade_porcentagem                        float64
Conteudo_gravimetrico_agua_kgkg1           float64
data_coleta_fmt                     datetime64[ns]
dtype: object

Antes de alterar as datas das outras tabelas, é necessário replicar os dados de data para todas as linhas onde a mesma foi feita o merge no xlsx.

In [82]:
loaded["Maturidade_fisiologica"]

,Data,Faixa,Parcela,Tratamento,Genotipo,Repeticao,Numero_graos,Numero_graos_maduros
0,2024-02-29,1,104,Disco,G1,1,30,0
1,NaT,1,104,Disco,G2,1,30,0
2,NaT,1,104,Disco,G3,1,30,0
3,NaT,1,107,Plantio Direto,G1,1,30,0
4,NaT,1,107,Plantio Direto,G2,1,30,0
...,...,...,...,...,...,...,...,...
211,NaT,3,309,Disco,G2,3,60,60
212,NaT,3,309,Disco,G3,3,60,60
213,NaT,3,310,Plantio Direto,G1,3,60,60
214,NaT,3,310,Plantio Direto,G2,3,60,60


In [133]:
fill_missing_dates(loaded["Maturidade_fisiologica"], "Data", "Data_fmt")

,Data,Faixa,Parcela,Tratamento,Genotipo,Repeticao,Numero_graos,Numero_graos_maduros,Data_fmt
0,2024-02-29,1,104,Disco,G1,1,30,0,2024-02-29
1,NaT,1,104,Disco,G2,1,30,0,2024-02-29
2,NaT,1,104,Disco,G3,1,30,0,2024-02-29
3,NaT,1,107,Plantio Direto,G1,1,30,0,2024-02-29
4,NaT,1,107,Plantio Direto,G2,1,30,0,2024-02-29
...,...,...,...,...,...,...,...,...,...
211,NaT,3,309,Disco,G2,3,60,60,2024-04-01
212,NaT,3,309,Disco,G3,3,60,60,2024-04-01
213,NaT,3,310,Plantio Direto,G1,3,60,60,2024-04-01
214,NaT,3,310,Plantio Direto,G2,3,60,60,2024-04-01


In [134]:
fill_missing_dates(loaded["Fenologia_completa"], "Data", "Data_fmt")

,Data,Faixa,Parcela,Tratamento,Genotipo,Repeticao,Planta,FV,FT,FB,Boneca,Unnamed: 11,Data_fmt
0,2024-01-15,1.0,1.0,Disco,NaN,R1,1,20,20,OK,*,NaN,2024-01-15
1,NaT,NaN,NaN,NaN,NaN,NaN,2,20,21,NaN,*,NaN,2024-01-15
2,NaT,NaN,NaN,NaN,NaN,NaN,3,21,23,NaN,*,NaN,2024-01-15
3,NaT,NaN,NaN,NaN,NaN,NaN,4,21,22,NaN,*,NaN,2024-01-15
4,NaT,NaN,NaN,NaN,NaN,NaN,5,21,22,NaN,*,NaN,2024-01-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,NaT,NaN,NaN,NaN,NaN,R2,6,-,-,-,-,OK,2024-01-22
76,NaT,NaN,NaN,NaN,NaN,NaN,7,-,-,-,-,OK,2024-01-22
77,NaT,NaN,NaN,NaN,NaN,NaN,8,-,-,-,-,OK,2024-01-22
78,NaT,NaN,NaN,NaN,NaN,NaN,9,-,-,-,-,-,2024-01-22


In [135]:
fill_missing_dates(loaded["Fenologia_inicial"], "Data", "Data_fmt")

,Data,Faixa,Parcela,Tratamento,Repeticao,Planta,Folhas_verdadeiras,Folhas_totais,Data_fmt
0,2023-12-14,1.0,1.0,Disco,R1,1,6,12,2023-12-14
1,NaT,NaN,NaN,NaN,NaN,2,7,12,2023-12-14
2,NaT,NaN,NaN,NaN,NaN,3,8,13,2023-12-14
3,NaT,NaN,NaN,NaN,NaN,4,7,13,2023-12-14
4,NaT,NaN,NaN,NaN,NaN,5,8,14,2023-12-14
...,...,...,...,...,...,...,...,...,...
135,NaT,NaN,NaN,NaN,R2,6,18,20,2024-01-11
136,NaT,NaN,NaN,NaN,NaN,7,17,19,2024-01-11
137,NaT,NaN,NaN,NaN,NaN,8,18,20,2024-01-11
138,NaT,NaN,NaN,NaN,NaN,9,17,19,2024-01-11


Além da data, existem campos string que também seguem o mesmo comportamento.

![pesos](../docs/pesos_columns.png)

In [136]:
fill_forward(loaded["Pesos"], "Preparo_solo", "Preparo_solo_fmt")

,Preparo_solo,Parcela,Identificacao,Plantas_por_metro,Peso_verde_5_plantas,Peso_verde_colmo_5_plantas,Peso_seco_colmo_5_plantas,Peso_verde_folhas_5_plantas,Peso_seco_folhas_5_plantas,Numero_de_espigas,Peso_verde_espigas,Peso_espiga_com_graos,Peso_verde_palha_espiga,Peso_seco_palha_espiga,Peso_verde_sabugo_espiga,Peso_seco_sabugo_espiga,Peso_verde_grao_espiga,Peso_seco_grao_espiga,Numero_bonecas,Peso_verde_bonecas,Peso_verde_palha_boneca,Peso_seco_palha_boneca,Peso_verde_sabugo_boneca,Peso_seco_sabugo_boneca,Peso_verde_graos_boneca,Peso_seco_graos_boneca,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Preparo_solo_fmt
0,Disco,104.0,45393.0,5.0,3475.0,1796.36,518.80,351.92,356.92,5.0,1498.30,1354.68,141.39,121.64,182.15,133.13,1126.62,1009.13,1.0,2.87,2.87,1.45,9999.00,9999.0,9999.0,9999.0,NaN,Preparo_solo,Parcela,Identificacao,Plantas_por_metro,Peso_verde_plantas,Peso_colmo_plantas,Peso_folhas_plantas,Area_foliar_plantas_cm2,Peso_verde_folhas_aliquota,Peso_seco_folhas_aliquota,Peso_verde_F_maduras,Peso_seco_F_maduras,Peso_verde_colmo_aliquota,Peso_seco_colmo_aliquota,Numero_espigas,Peso_verde_espiga,Peso_espiga_com_graos,Peso_verde_palha_da_espiga,Peso_seco_palha_da_espiga,Peso_verde_sabugo,Peso_seco_sabugo,Peso_verde_graos,Peso_seco_graos,Numero_bonecas,Peso_verde_bonecas,Peso_verde_palha_boneca,Peso_seco_palha_boneca,Peso_verde_sabugo_boneca,Peso_seco_sabugo_boneca,Disco
1,NaN,211.0,45393.0,5.0,2745.0,1174.09,438.61,311.81,311.81,5.0,1225.85,1101.13,122.08,104.99,142.61,114.22,957.04,831.15,3.0,17.11,13.80,1.98,3.52,2.1,9999.0,9999.0,NaN,Disco,104,45279,5,1700,1021.98,690.42,18009.6,355,97.6,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,Disco
2,NaN,309.0,45393.0,5.0,3390.0,1514.31,507.22,352.45,352.45,5.0,1485.47,1336.19,145.96,121.79,187.52,142.15,1150.62,995.19,1.0,2.14,2.14,0.91,9999.00,9999.0,9999.0,9999.0,NaN,NaN,NaN,45303,5,4440,3124,1052,39622.51,330.53,57.53,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,Disco
3,Plantio Direto,107.0,45393.0,5.0,3065.0,1577.90,408.15,324.24,324.24,5.0,1134.39,1014.56,117.55,94.20,113.62,101.21,883.13,760.05,1.0,1.76,1.76,0.10,9999.00,9999.0,9999.0,9999.0,NaN,NaN,NaN,45348,5,6790,2920,885,35619.87,885,276.16,47.45,36.86,1130,280.4,6,2358.73,1882.12,442.63,148.59,426.64,167.87,1440.89,770.37,2,38.8,40.37,7.53,10.89,0.66,Plantio Direto
4,NaN,204.0,45393.0,5.0,2970.0,1246.17,364.62,302.80,302.80,5.0,1402.43,1291.98,108.04,93.08,163.38,120.24,1126.76,968.43,1.0,1.48,1.48,0.01,9999.00,9999.0,9999.0,9999.0,NaN,NaN,211,45279,5,1800,1134,708.19,16798.92,421.01,88.3,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,Plantio Direto
5,NaN,310.0,45393.0,5.0,2730.0,1158.13,436.12,228.96,430.41,5.0,1325.06,1209.44,113.39,107.11,164.79,128.61,1045.67,903.36,2.0,5.75,5.75,3.79,9999.00,9999.0,9999.0,9999.0,NaN,NaN,NaN,45303,5,5040,3140,1090,46643.47,358.96,71.31,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,Plantio Direto
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45348,5,6020,2450,720,28992.35,720,247.5,21.85,18.72,785.33,209.04,5,2142.94,1787.49,323.55,124.5,399.34,163.02,1377.69,744.58,3,99.41,58.88,12.12,16,1.5,Plantio Direto
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,309,45279,5,733.02,389.64,342.37,8157.76,342.37,48.4,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,9999,Plantio Direto
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [137]:
fill_forward(loaded["Fenologia_completa"], "Faixa ", "Faixa_fmt")
fill_forward(loaded["Fenologia_completa"], "Parcela", "Parcela_fmt")
fill_forward(loaded["Fenologia_completa"], "Tratamento", "Tratamento_fmt")
fill_forward(loaded["Fenologia_completa"], "Repeticao", "Repeticao_fmt")
loaded["Fenologia_completa"]

,Data,Faixa,Parcela,Tratamento,Genotipo,Repeticao,Planta,FV,FT,FB,Boneca,Unnamed: 11,Data_fmt,Faixa_fmt,Parcela_fmt,Tratamento_fmt,Repeticao_fmt
0,2024-01-15,1.0,1.0,Disco,NaN,R1,1,20,20,OK,*,NaN,2024-01-15,1.0,1.0,Disco,R1
1,NaT,NaN,NaN,NaN,NaN,NaN,2,20,21,NaN,*,NaN,2024-01-15,1.0,1.0,Disco,R1
2,NaT,NaN,NaN,NaN,NaN,NaN,3,21,23,NaN,*,NaN,2024-01-15,1.0,1.0,Disco,R1
3,NaT,NaN,NaN,NaN,NaN,NaN,4,21,22,NaN,*,NaN,2024-01-15,1.0,1.0,Disco,R1
4,NaT,NaN,NaN,NaN,NaN,NaN,5,21,22,NaN,*,NaN,2024-01-15,1.0,1.0,Disco,R1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,NaT,NaN,NaN,NaN,NaN,R2,6,-,-,-,-,OK,2024-01-22,2.0,2.0,Plantio Direto,R2
76,NaT,NaN,NaN,NaN,NaN,NaN,7,-,-,-,-,OK,2024-01-22,2.0,2.0,Plantio Direto,R2
77,NaT,NaN,NaN,NaN,NaN,NaN,8,-,-,-,-,OK,2024-01-22,2.0,2.0,Plantio Direto,R2
78,NaT,NaN,NaN,NaN,NaN,NaN,9,-,-,-,-,-,2024-01-22,2.0,2.0,Plantio Direto,R2


In [138]:
fill_forward(loaded["Fenologia_inicial"], "Faixa ", "Faixa_fmt")
fill_forward(loaded["Fenologia_inicial"], "Parcela", "Parcela_fmt")
fill_forward(loaded["Fenologia_inicial"], "Tratamento", "Tratamento_fmt")
fill_forward(loaded["Fenologia_inicial"], "Repeticao", "Repeticao_fmt")
loaded["Fenologia_inicial"]

,Data,Faixa,Parcela,Tratamento,Repeticao,Planta,Folhas_verdadeiras,Folhas_totais,Data_fmt,Faixa_fmt,Parcela_fmt,Tratamento_fmt,Repeticao_fmt
0,2023-12-14,1.0,1.0,Disco,R1,1,6,12,2023-12-14,1.0,1.0,Disco,R1
1,NaT,NaN,NaN,NaN,NaN,2,7,12,2023-12-14,1.0,1.0,Disco,R1
2,NaT,NaN,NaN,NaN,NaN,3,8,13,2023-12-14,1.0,1.0,Disco,R1
3,NaT,NaN,NaN,NaN,NaN,4,7,13,2023-12-14,1.0,1.0,Disco,R1
4,NaT,NaN,NaN,NaN,NaN,5,8,14,2023-12-14,1.0,1.0,Disco,R1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,NaT,NaN,NaN,NaN,R2,6,18,20,2024-01-11,2.0,2.0,Plantio Direto,R2
136,NaT,NaN,NaN,NaN,NaN,7,17,19,2024-01-11,2.0,2.0,Plantio Direto,R2
137,NaT,NaN,NaN,NaN,NaN,8,18,20,2024-01-11,2.0,2.0,Plantio Direto,R2
138,NaT,NaN,NaN,NaN,NaN,9,17,19,2024-01-11,2.0,2.0,Plantio Direto,R2


## Análise de dados

Considerando que a base está correta (ainda não está uniforme em relação as colunas), porém seus dados estão condizentes com os valores do xlsx, começa a avaliação das tabelas as quais vão servir de base para a identificação do stress hídrico das plantas.

"Um dos principais fatores que provocam estresse nas plantas e afetam sobremaneira a produtividade e persistência de espécies forrageiras é causado pelo déficit hídrico. É comum ocorrer déficit hídrico durante o crescimento das plantas, tendo como consequência a redução temporária do crescimento e da biomassa. Déficits hídricos em plantas ocorrem principalmente durante a estação de seca, quando a água do solo não está disponível para a planta por dias, semanas ou até meses, causando um decréscimo nas atividades fisiológicas."(Cavalcante, Ana; 2009)

source: https://www.infoteca.cnptia.embrapa.br/bitstream/doc/748148/1/doc89.pdf

Considerando que um dos fatores é a falta de água no solo, uma das principais tabelas de atuação será a de Umidade do solo.
Para isso primeiro é necessário entender o que cada coluna significa:

In [102]:
loaded["Umidade_de_solo"].columns

Index(['Profundidade_cm', 'Registro', 'Identificacao', 'Data_coleta', 'Material', 'Umidade_porcentagem',
       'Conteudo_gravimetrico_agua_kgkg1', 'data_coleta_fmt'],
      dtype='object')

Profundidade_cm: Profundidade em centimetros que foi coletada uma amostra da terra.

Registro: Somente um ID de identificação do registro.

Identificação: Em qual plantação foi coletada essa informação (Solo D1) = Solo do Disco 1

Data_coleta: Quando foi coletado

Material: Qual o material coletado

Umidade_porcentagem: A porcentagem da umidade do solo.

Conteudo_gravimetrico_agua_kgkg1 = CGA – conteúdo gravimétrico de água (umidade em base gravimétrica), em kg kg^-1

Source: https://www.infoteca.cnptia.embrapa.br/infoteca/bitstream/doc/1087253/1/Pt1Cap2Umidadeatual.pdf

Para simplificar a forma que será incluida a tabela de Umidade em outra tabelas é valido se pivotar as profundidades, removendo a duplicação das linhas das outra tabelas ao se realizar o join.

In [148]:
# Assuming 'df_umidade_solo' is your loaded and cleaned dataframe
# We pivot the table to make each 'Profundidade' a separate column
df_solo_wide = loaded["Umidade_de_solo"].pivot_table(
    index=['data_coleta_fmt', 'Registro', 'Identificacao', 'Material'],
    columns='Profundidade_cm',
    values=['Conteudo_gravimetrico_agua_kgkg1', 'Umidade_porcentagem']
).reset_index()

df_solo_wide.columns.name = None

print("Pivoted Soil Moisture Data:")
df_solo_wide

Pivoted Soil Moisture Data:


data_coleta_fmt Registro Identificacao Material Conteudo_gravimetrico_agua_kgkg1                  \
Profundidade_cm                                                                             0-30   30-60   60-90   
0                    2023-12-21   425757       Solo D1     SOLO                           0.2579     NaN     NaN   
1                    2023-12-21   425758       Solo D2     SOLO                              NaN  0.2792     NaN   
2                    2023-12-21   425759       Solo D3     SOLO                              NaN     NaN  0.2930   
3                    2023-12-21   425760       Solo D4     SOLO                              NaN     NaN     NaN   
4                    2023-12-21   425761      Solo PD1     SOLO                           0.2763     NaN     NaN   
5                    2023-12-21   425762      Solo PD2     SOLO                              NaN  0.3056     NaN   
6                    2023-12-21   425763      Solo PD3     SOLO                              NaN     NaN  0.3020   
7                    2023-12-21   425764      Solo PD4     SOLO                              NaN     NaN     NaN   
8                    2023-12-28   425765      Solo D1      SOLO                           0.2722     NaN     NaN   
9                    2023-12-28   425766       Solo D2     SOLO                              NaN  0.2694     NaN   
10                   2023-12-28   425767       Solo D3     SOLO                              NaN     NaN  0.2872   
11                   2023-12-28   425768       Solo D4     SOLO                              NaN     NaN     NaN   
12                   2023-12-28   425769      Solo PD1     SOLO                           0.2701     NaN     NaN   
13                   2023-12-28   425770      Solo PD2     SOLO                              NaN  0.2882     NaN   
14                   2023-12-28   425771      Solo PD3     SOLO                              NaN     NaN  0.2952   
15                   2023-12-28   425772      Solo PD4     SOLO                              NaN     NaN     NaN   
16                   2024-06-18     9999      Solo D1      SOLO                           0.1388     NaN     NaN   
17                   2024-06-18     9999       Solo D2     SOLO                              NaN  0.1581     NaN   
18                   2024-06-18     9999       Solo D3     SOLO                              NaN     NaN  0.1678   
19                   2024-06-18     9999       Solo D4     SOLO                              NaN     NaN     NaN   
20                   2024-06-18     9999      Solo PD1     SOLO                           0.1457     NaN     NaN   
21                   2024-06-18     9999      Solo PD2     SOLO                              NaN  0.1700     NaN   
22                   2024-06-18     9999      Solo PD3     SOLO                              NaN     NaN  0.1768   
23                   2024-06-18     9999      Solo PD4     SOLO                              NaN     NaN     NaN   

                        Umidade_porcentagem                       
Profundidade_cm  90-120                0-30  30-60  60-90 90-120  
0                   NaN               25.79    NaN    NaN    NaN  
1                   NaN                 NaN  27.92    NaN    NaN  
2                   NaN                 NaN    NaN  29.30    NaN  
3                0.2889                 NaN    NaN    NaN  28.89  
4                   NaN               27.63    NaN    NaN    NaN  
5                   NaN                 NaN  30.56    NaN    NaN  
6                   NaN                 NaN    NaN  30.20    NaN  
7                0.2978                 NaN    NaN    NaN  29.78  
8                   NaN               27.22    NaN    NaN    NaN  
9                   NaN                 NaN  26.94    NaN    NaN  
10                  NaN                 NaN    NaN  28.72    NaN  
11               0.2874                 NaN    NaN    NaN  28.74  
12                  NaN               27.01    NaN    NaN    

Para realizar o join entre as tabelas é necessário avaliar quais possuem esses valores.

Tabelas que possuem data como coluna:
- Umidade_de_solo
- Umidade_de_graos_maturidade
- Fenologia_inicial
- Fenologia_completa
- Maturidade_fisiologica

Tabelas que possuem faixa e parcela como colunas possiveis de join:
- Espiga
- Dados de colheita
- Altura planta
- Pesos

Avaliação das datas das tabelas

In [126]:
sorted(df_solo_wide["data_coleta_fmt"].unique())

[Timestamp('2023-12-21 00:00:00'),
 Timestamp('2023-12-28 00:00:00'),
 Timestamp('2024-06-18 00:00:00')]

In [123]:
sorted(loaded["Umidade_de_graos_maturidade"]["Data_entrada"].unique())

[Timestamp('2024-04-01 00:00:00'),
 Timestamp('2024-04-08 00:00:00'),
 Timestamp('2024-04-12 00:00:00'),
 Timestamp('2024-04-18 00:00:00')]

In [125]:
sorted(loaded["Fenologia_inicial"]["Data_fmt"].unique())

[Timestamp('2023-12-14 00:00:00'),
 Timestamp('2023-12-18 00:00:00'),
 Timestamp('2023-12-22 00:00:00'),
 Timestamp('2023-12-26 00:00:00'),
 Timestamp('2023-12-28 00:00:00'),
 Timestamp('2024-01-08 00:00:00'),
 Timestamp('2024-01-11 00:00:00')]

In [124]:
sorted(loaded["Fenologia_completa"]["Data_fmt"].unique())

[Timestamp('2024-01-15 00:00:00'),
 Timestamp('2024-01-17 00:00:00'),
 Timestamp('2024-01-19 00:00:00'),
 Timestamp('2024-01-22 00:00:00')]

In [127]:
sorted(loaded["Maturidade_fisiologica"]["Data_fmt"].unique())

[Timestamp('2024-02-29 00:00:00'),
 Timestamp('2024-03-06 00:00:00'),
 Timestamp('2024-03-08 00:00:00'),
 Timestamp('2024-03-11 00:00:00'),
 Timestamp('2024-03-13 00:00:00'),
 Timestamp('2024-03-15 00:00:00'),
 Timestamp('2024-03-18 00:00:00'),
 Timestamp('2024-03-20 00:00:00'),
 Timestamp('2024-03-22 00:00:00'),
 Timestamp('2024-03-25 00:00:00'),
 Timestamp('2024-03-27 00:00:00'),
 Timestamp('2024-04-01 00:00:00')]

Dentro das tabelas que possuem datas, as datas não são identicas, portanto não dá para concluir se elas são passíveis de serem unidas sem uma revisão e ajuste das datas por período, ou outro ajuste validando com o especialista

In [157]:
loaded["Espigas"].shape

(18, 11)

In [160]:
loaded["Altura_plantas"].shape

(18, 15)

In [161]:
loaded["Pesos"].shape

(10, 57)

In [162]:
loaded["Dados_de_colheita"].shape

(171, 11)

In [166]:
loaded["Pesos"]["Parcela"].unique()

array([104., 211., 309., 107., 204., 310.,  nan])

In [165]:
loaded["Altura_plantas"]["Parcela"].unique()

array([104, 107, 204, 211, 309, 310], dtype=int64)

In [163]:
loaded["Espigas"]["Parcela"].unique()

array([104, 107, 204, 211, 309, 310], dtype=int64)

In [164]:
loaded["Dados_de_colheita"]["Parcela"].unique()

array([ 101,  102,  103,  104,  105,  106,  107,  108,  109,  110,  111,
        201,  202,  203,  204,  205,  206,  207,  208,  209,  210,  211,
        301,  302,  303,  304,  305,  306,  307,  308,  309,  310,  311,
        401,  402,  403,  404,  405,  406,  407,  408,  409,  410,  411,
        501,  502,  503,  504,  505,  506,  507,  508,  509,  510,  511,
        601,  602,  603,  604,  605,  606,  607,  608,  609,  610,  611,
        701,  702,  703,  704,  705,  706,  707,  708,  709,  710,  711,
        801,  802,  803,  804,  805,  806,  807,  808,  809,  810,  811,
        901,  902,  903,  904,  905,  906,  907,  908,  909,  910,  911,
       1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1201, 1202, 1203,
       1204, 1205, 1206, 1207, 1208, 1301, 1302, 1303, 1304, 1305, 1306,
       1307, 1308, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1501,
       1502, 1503, 1504, 1505, 1506, 1507, 1508, 1601, 1602, 1603, 1604,
       1605, 1606, 1607, 1608, 1701, 1702, 1703, 17

Ao analisar as tabelas com faixa e parcela, pode-se notar que os Dados de colheita possuem muito mais parcelas que as outras 3 tabelas, possivelmente as tabelas de Espiga/Altura_planta e Pesos sejam amostras de um todo, portanto possuem partes dos valores

As datas das tabelas não batem, e como não existe um identificador único não há como juntar as tabelas por esses quesitos, sendo necessário repensar a forma de estruturar a tabela.

# Padronização de nomes de colunas (snake_case, sem acentos)
# Este bloco define utilitários e aplica a padronização a todas as tabelas carregadas

In [147]:
# Aplicar padronização a todas as tabelas carregadas e derivadas, após as transformações acima
loaded_std = {}
for k, df in loaded.items():
    loaded_std[k] = standardize_columns(df) if df is not None else None
loaded_std["Umidade_de_solo"]

,profundidade_cm,registro,identificacao,data_coleta,material,umidade_porcentagem,conteudo_gravimetrico_agua_kgkg1,data_coleta_fmt
0,0-30,425757,Solo D1,21/12/2023,SOLO,25.79,0.2579,2023-12-21
1,30-60,425758,Solo D2,21/12/2023,SOLO,27.92,0.2792,2023-12-21
2,60-90,425759,Solo D3,21/12/2023,SOLO,29.30,0.2930,2023-12-21
3,90-120,425760,Solo D4,21/12/2023,SOLO,28.89,0.2889,2023-12-21
4,0-30,425761,Solo PD1,21/12/2023,SOLO,27.63,0.2763,2023-12-21
5,30-60,425762,Solo PD2,21/12/2023,SOLO,30.56,0.3056,2023-12-21
6,60-90,425763,Solo PD3,21/12/2023,SOLO,30.20,0.3020,2023-12-21
7,90-120,425764,Solo PD4,21/12/2023,SOLO,29.78,0.2978,2023-12-21
8,0-30,425765,Solo D1,28/12/2023,SOLO,27.22,0.2722,2023-12-28
9,30-60,425766,Solo D2,2023-12-28 00:00:00,SOLO,26.94,0.2694,2023-12-28


In [175]:
loaded["Espigas"]
loaded["Altura_plantas"]
loaded["Pesos"]
# Faixa/Parcela


loaded["Dados_de_colheita"]
#Parcela

#Data
loaded["Umidade_de_solo"]
loaded["Umidade_de_graos_maturidade"]
loaded["Fenologia_inicial"]
loaded["Fenologia_completa"]
loaded["Maturidade_fisiologica"]


,Data,Faixa,Parcela,Tratamento,Genotipo,Repeticao,Numero_graos,Numero_graos_maduros,Data_fmt
0,2024-02-29,1,104,Disco,G1,1,30,0,2024-02-29
1,NaT,1,104,Disco,G2,1,30,0,2024-02-29
2,NaT,1,104,Disco,G3,1,30,0,2024-02-29
3,NaT,1,107,Plantio Direto,G1,1,30,0,2024-02-29
4,NaT,1,107,Plantio Direto,G2,1,30,0,2024-02-29
...,...,...,...,...,...,...,...,...,...
211,NaT,3,309,Disco,G2,3,60,60,2024-04-01
212,NaT,3,309,Disco,G3,3,60,60,2024-04-01
213,NaT,3,310,Plantio Direto,G1,3,60,60,2024-04-01
214,NaT,3,310,Plantio Direto,G2,3,60,60,2024-04-01
